In [ ]:
import pandas as pd
import ixmp as ix
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ix.Platform(dbtype='HSQLDB')

In [ ]:
base = message_ix.Scenario(mp, model='Westeros Electrified', 
                               scen='baseline')

In [ ]:
model = 'Westeros Electrified'
scen = base.clone(model, 'renewables','testing new formulation', keep_sol=False)
scen.check_out()

vintage_years, act_years = scen.vintage_and_active_years()
model_horizon = scen.set('year')
country = 'Westeros'

## add Renewbale Formulation
### starting with sets
- level_renewable ... commodity level from which the the technologies that shall be considered renewable take their input from

- grade ... "quality" of renewable potential

- rating ... "share" of renewable generation that contributes to reliability

#### level_renewable 
The wind_ppl now has the commodity *wind* on level *renewable* as input

### Load Factors - Describing the Electricity Sector
#### Peak load and flexibility load factors
If we further specify our demand, we will find that the demand for electricity (!not the demand for useful light but for electricity!) has a peak load that is different from the annual average load; e.g: if annual average load is 5 GWh peak load during midday might be 10GW --> the peak load factor is equal to 2. Additionally, our demand is not flexible, but it much rather requires flexibility from the grid. Thus, it has a flexibility factor of eg. 10%. 


In [ ]:
# peak_load_factor(node,commodity,level,year,time)
peak_load_factor = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary',       
        'year': model_horizon,
        'time' : 'year',
        'value' : 2,
        'unit' : '???'})

scen.add_par('peak_load_factor', peak_load_factor)

### Renewable Potential 
#### grades, renewable potential and the capacity factor
To supply the wind ppl we assume a wind potential of 1000 GW at a capacity factor of 0.7 and 10000 GW at less good locations with acapacity factors of 0.1

## Sescribing the Renewable Technologies Reliability and Flexibility Behaviour
### rating and reliability --> Ensuring enough Firm Capacity
we assume, that up to a share of 10% of total electricity supply, the capacity of wind_ppl adds by 80% to the firm capacity of the power system (r80).
The remaining 90% installed capacity, however  only  5% contribute to the firm capcity (r5).


In [ ]:
# add the ratings as a set 
scen.add_set('rating', ['r1', 'r2'])

# define the share of commodity used (10% and 90%)
base_reliability = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary', 
        'unit': '???',
        'time': 'year',
        'year_act': model_horizon})

# for Wind PPL
rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.1, rating= 'r1')
scen.add_par('rating_bin', rating_bin)

reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 1, rating= 'r1')
scen.add_par('reliability_factor', reliability_factor)

rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.9, rating= 'r2')
scen.add_par('rating_bin', rating_bin)

reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 0.5, rating= 'r2')
scen.add_par('reliability_factor', reliability_factor)

reliability_factor = make_df(base_reliability, technology= 'coal_ppl', value = 1, rating= 'firm')
scen.add_par('reliability_factor', reliability_factor)

### flexibility of demand and supply --> ensuring the activity flexibility reserve 

The wind power plant has a flexibility demand of 5% of its ACT. The coal powerplant can provide 20% of it's activity as flexibility.

### commit and solve

In [ ]:
scen.commit(comment='define parameters for renewable implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

### plotting

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700)

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
#p.plot_prices(subset=['light'], baseyear=True)

In [ ]:
mp.close_db()